In [2]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
# Make sure you have the right PyTorch version
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# 3. Install PyTorch Geometric CPU-only wheels
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.9.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.9.0+cpu.html
!pip install torch-geometric-temporal

2.9.0+cu128
Looking in indexes: https://download.pytorch.org/whl/cpu
Looking in links: https://data.pyg.org/whl/torch-2.9.0+cpu.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "/home/hongoo/Projects/hacx/hacx/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "/home/hongoo/Projects/hacx/hacx/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/hongoo/Projects/hacx/hacx/lib/python3.13/site-pac

In [3]:
!pip install numpy --upgrade


In [4]:
!pip install requests

In [5]:
import torch
import numpy as np
import pandas as pd

print(f"torch version: {torch.__version__}")

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(edgeitems=3, precision=3)
torch.set_printoptions(edgeitems=2, precision=3)

# Utility functions ################
def print_matrix(matrix):
    return pd.DataFrame(matrix)

def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

torch version: 2.9.0+cu128


## DATASET Extraction from data.gov.sg


In [6]:
import requests

dataset_id_mrt = "d_b39d3a0871985372d7e1637193335da5"
poll_url = f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id_mrt}/poll-download"

# Step 1: Poll for download URL
resp = requests.get(poll_url)
data = resp.json()
if data['code'] != 0:
    print("Error:", data['errMsg'])
    exit(1)

download_url = data['data']['url']

# Step 2: Download and save locally
with requests.get(download_url, stream=True) as r:
    r.raise_for_status()
    with open("mrt_dataset.json", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("MRT dataset saved as mrt_dataset.json")


MRT dataset saved as mrt_dataset.json


In [7]:
import requests

dataset_id_parks = "d_77d7ec97be83d44f61b85454f844382f"
poll_url = f"https://api-open.data.gov.sg/v1/public/api/datasets/{dataset_id_parks}/poll-download"

# Step 1: Poll download
resp = requests.get(poll_url)
data = resp.json()
if data['code'] != 0:
    print("Error:", data['errMsg'])
    exit(1)

download_url = data['data']['url']

# Step 2: Download actual dataset
with requests.get(download_url, stream=True) as r:
    r.raise_for_status()
    with open("parks_dataset.json", "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print("Dataset saved to parks_dataset.json")


Dataset saved to parks_dataset.json


In [31]:
import json

# Load MRT dataset
with open("mrt_dataset.json", "r", encoding="utf-8") as f:
    mrt_data = json.load(f)

# Load parks dataset
with open("parks_dataset.json", "r", encoding="utf-8") as f:
    parks_data = json.load(f)


In [8]:
import json

# Load the full GeoJSON FeatureCollection
with open("parks_dataset.json", "r", encoding="utf-8") as f:
    parks_data = json.load(f)

# Target park name
target_name = "WEST COAST PARK"

# Find the feature that matches (case-insensitive)
west_coast_feature = next(
    (f for f in parks_data["features"]
     if target_name.lower() in f["properties"].get("NAME", "").lower()),
    None
)

# Check and save result
if west_coast_feature:
    print("✅ Found:", west_coast_feature["properties"]["NAME"])
    with open("west_coast_park.json", "w", encoding="utf-8") as f:
        json.dump(west_coast_feature, f, indent=4)
else:
    print("❌ No park found with name containing:", target_name)


✅ Found: WEST COAST PARK


Park Name: WEST COAST PLACE PLAYGROUND
Area: None
Perimeter: None
Centroid (lon, lat): (np.float64(103.75841745648589), np.float64(1.3144082681383233))


In [ ]:
import requests
import pandas as pd
import os
from datetime import datetime

# Directory to save CSV
save_dir = "2025_hol"
os.makedirs(save_dir, exist_ok=True)

dataset_id = "d_3751791452397f1b1c80c451447e40b7"
url = f"https://data.gov.sg/api/action/datastore_search?resource_id={dataset_id}&limit=1000"

response = requests.get(url)
data = response.json()

# Convert records to DataFrame
records = data['result']['records']
df = pd.DataFrame(records)
print(df.columns)
# Convert to datetime
df['date'] = pd.to_datetime(df['date'])

# Filter for 2025
df_2025 = df[df['date'].dt.year == 2025]

csv_path = os.path.join(save_dir, "holidays_2025.csv")
df_2025.to_csv(csv_path, index=False)
print(f"Saved CSV to {csv_path}")



Index(['_id', 'date', 'day', 'holiday'], dtype='object')
Saved CSV to 2025_hol/holidays_2025.csv


In [ ]:
import requests
          
collection_id = 1459          
url = "https://api-production.data.gov.sg/v2/public/api/collections/{}/metadata".format(collection_id)
        
response = requests.get(url)
print(response.json())


{'code': 0, 'data': {'collectionMetadata': {'collectionId': '1459', 'createdAt': '2016-12-14T17:02:44+08:00', 'name': 'Realtime Weather Readings across Singapore', 'description': 'NEA provides APIs for readings of temperature, humidity, precipitation and wind conditions at up to one-minute intervals. The data is provided at weather-station level. ', 'lastUpdatedAt': '2025-08-01T14:53:47+08:00', 'coverageStart': '2016-12-14T08:00:00+08:00', 'frequency': 'realtime', 'sources': ['National Environment Agency'], 'managedBy': 'National Environment Agency', 'childDatasets': ['d_66b77726bbae1b33f218db60ff5861f0', 'd_6580738cdd7db79374ed3152159fbd69', 'd_2d3b0c4da128a9a59efca806441e1429', 'd_534cf203023b51f51f879145ccc56ff9', 'd_7677738484067741bf3b56ab5d69c7e9']}}, 'errorMsg': ''}


In [ ]:
import json
from bs4 import BeautifulSoup  # pip install beautifulsoup4

# Load GeoJSON file
with open("mrt_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Check if top-level is a FeatureCollection
if "features" not in data:
    raise ValueError("JSON file is not a valid GeoJSON FeatureCollection")

# Extract station names and coordinates
geospatial_data = []

for feature in data["features"]:
    properties = feature.get("properties", {})
    geometry = feature.get("geometry", {})
    coords = geometry.get("coordinates", [])

    # Parse the Description HTML to get STATION_NA
    description = properties.get("Description", "")
    soup = BeautifulSoup(description, "html.parser")
    table = soup.find("table")
    
    station_name = None
    if table:
        for row in table.find_all("tr"):
            th = row.find("th")
            td = row.find("td")
            if th and th.text.strip() == "STATION_NA":
                station_name = td.text.strip()
                break
    
    geospatial_data.append({
        "name": station_name,
        "coordinates": coords
    })

# Print results
for item in geospatial_data:
    print(item)


{'name': 'KAKI BUKIT MRT STATION', 'coordinates': [103.909146276263, 1.33492189034272, 0.0]}
{'name': 'BEDOK RESERVOIR MRT STATION', 'coordinates': [103.933487189855, 1.33655490332863, 0.0]}
{'name': 'BENCOOLEN MRT STATION', 'coordinates': [103.849271703873, 1.29769911765443, 0.0]}
{'name': 'BENCOOLEN MRT STATION', 'coordinates': [103.850843384717, 1.29919519603019, 0.0]}
{'name': 'KAKI BUKIT MRT STATION', 'coordinates': [103.909405098626, 1.33531120636956, 0.0]}
{'name': 'TAMPINES WEST MRT STATION', 'coordinates': [103.938948503782, 1.34499919751448, 0.0]}
{'name': 'JALAN BESAR MRT STATION', 'coordinates': [103.855296125414, 1.30488989412929, 0.0]}
{'name': 'BENDEMEER MRT STATION', 'coordinates': [103.863074005937, 1.31385250562225, 0.0]}
{'name': 'BEDOK NORTH MRT STATION', 'coordinates': [103.917810526029, 1.33442257655151, 0.0]}
{'name': 'UPPER CHANGI MRT STATION', 'coordinates': [103.961043080709, 1.34084047406433, 0.0]}
{'name': 'GEYLANG BAHRU MRT STATION', 'coordinates': [103.871

In [ ]:
# Filter for Kent Ridge MRT Station
kent_ridge = [item for item in geospatial_data if item["name"] == "KENT RIDGE MRT STATION"]

# Print result
for item in kent_ridge:
    print(item)


{'name': 'KENT RIDGE MRT STATION', 'coordinates': [103.784809631882, 1.29242811590855, 0.0]}
{'name': 'KENT RIDGE MRT STATION', 'coordinates': [103.784465895365, 1.29431588700949, 0.0]}
{'name': 'KENT RIDGE MRT STATION', 'coordinates': [103.784490451682, 1.29398478516701, 0.0]}
{'name': 'KENT RIDGE MRT STATION', 'coordinates': [103.784534656639, 1.29321880284117, 0.0]}


In [1]:
import folium
import json
import numpy as np
import os

# Load the GeoJSON feature (falls back to file if needed)
if 'west_coast' not in globals():
    if os.path.exists("west_coast_park.json"):
        with open("west_coast_park.json", "r", encoding="utf-8") as f:
            west_coast = json.load(f)
    else:
        raise NameError("west_coast not found in memory and no west_coast_park.json available. Run the extraction cell first.")

# Load MRT data if not already in memory
if 'geospatial_data' not in globals():
    with open("mrt_dataset.json", "r", encoding="utf-8") as f:
        mrt_data = json.load(f)
        
        # Extract station data like in the previous cell
        geospatial_data = []
        from bs4 import BeautifulSoup
        
        for feature in mrt_data["features"]:
            properties = feature.get("properties", {})
            geometry = feature.get("geometry", {})
            coords = geometry.get("coordinates", [])
            
            description = properties.get("Description", "")
            soup = BeautifulSoup(description, "html.parser")
            table = soup.find("table")
            
            station_name = None
            if table:
                for row in table.find_all("tr"):
                    th = row.find("th")
                    td = row.find("td")
                    if th and th.text.strip() == "STATION_NA":
                        station_name = td.text.strip()
                        break
            
            if station_name and coords:  # Only add if we have both name and coordinates
                geospatial_data.append({
                    "name": station_name,
                    "coordinates": coords
                })

# Helper to flatten nested Polygon/MultiPolygon coords into [[lon, lat], ...]
def _flatten_coords(coords):
    out = []
    if not isinstance(coords, (list, tuple)):
        return out
    # coordinate pair case
    if len(coords) >= 2 and isinstance(coords[0], (int, float)) and isinstance(coords[1], (int, float)):
        return [coords[:2]]
    for c in coords:
        out.extend(_flatten_coords(c))
    return out

coords_pairs = _flatten_coords(west_coast.get("geometry", {}).get("coordinates", []))
if len(coords_pairs) == 0:
    raise ValueError("No coordinate pairs found in west_coast.geometry.coordinates")

coords_np = np.array(coords_pairs, dtype=float)
# GeoJSON: [lon, lat] -> folium expects [lat, lon]
centroid_lon = coords_np[:, 0].mean()
centroid_lat = coords_np[:, 1].mean()
center = [float(centroid_lat), float(centroid_lon)]

# Initialize map centered on West Coast Park
m = folium.Map(location=center, zoom_start=16)

# Add GeoJSON polygon layer
folium.GeoJson(
    west_coast,
    name="West Coast Park",
    style_function=lambda feature: {
        "fillColor": "green",
        "color": "darkgreen",
        "weight": 2,
        "fillOpacity": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(fields=["NAME"], aliases=["Feature Name:"])
).add_to(m)

# Add all Kent Ridge MRT stations as markers
kent_ridge_stations = [
    item for item in geospatial_data 
    if item["name"] and "KENT RIDGE" in item["name"].upper()
]

for station in kent_ridge_stations:
    coords = station["coordinates"]
    # GeoJSON uses [lon, lat], but folium needs [lat, lon]
    location = [coords[1], coords[0]]  # flip coordinates for folium
    folium.Marker(
        location=location,
        popup=station["name"],
        icon=folium.Icon(color="red", icon="train", prefix='fa')
    ).add_to(m)

# Save map
m.save("west_coast_park_map.html")
print(f"Found {len(kent_ridge_stations)} Kent Ridge related stations")
for station in kent_ridge_stations:
    print(f"Added marker for: {station['name']} at coordinates [lon={station['coordinates'][0]:.6f}, lat={station['coordinates'][1]:.6f}]")

Found 4 Kent Ridge related stations
Added marker for: KENT RIDGE MRT STATION at coordinates [lon=103.784810, lat=1.292428]
Added marker for: KENT RIDGE MRT STATION at coordinates [lon=103.784466, lat=1.294316]
Added marker for: KENT RIDGE MRT STATION at coordinates [lon=103.784490, lat=1.293985]
Added marker for: KENT RIDGE MRT STATION at coordinates [lon=103.784535, lat=1.293219]


In [ ]:
# -----------------------------
# 1. Prepare Data
# -----------------------------
# X: [num_nodes, seq_len, num_features]  --> past footfall, weather, time info
# y: [num_nodes, seq_len]               --> footfall target

X_train, y_train = load_data()  # load past West Coast Park data
X_test, y_test = load_data_test()

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.float)

# -----------------------------
# 2. Define STGNN Model
# -----------------------------
class STGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, seq_len):
        super(STGNN, self).__init__()

        # Spatial layer (optional for single node)
        self.gcn = GCNConv(input_dim, hidden_dim)

        # Temporal layer (LSTM)
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=hidden_dim, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index=None):
        # x: [num_nodes, seq_len, input_dim]
        # Apply GCN for spatial info (optional)
        if edge_index is not None:
            # Flatten time for GCN
            num_nodes, seq_len, feat_dim = x.shape
            x_flat = x.view(num_nodes*seq_len, feat_dim)
            x_gcn = self.gcn(x_flat, edge_index)
            x = x_gcn.view(num_nodes, seq_len, -1)
        else:
            # Pass directly to temporal
            x = x

        # Temporal LSTM
        lstm_out, _ = self.lstm(x)  # output: [num_nodes, seq_len, hidden_dim]

        # Predict next time step
        out = self.fc(lstm_out[:, -1, :])  # take last timestep
        return out

# -----------------------------
# 3. Train Model
# -----------------------------
model = STGNN(input_dim=X_train.shape[2], hidden_dim=64, output_dim=1, seq_len=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch}, Loss: {loss.item()}")

# -----------------------------
# 4. Predict
# -----------------------------
model.eval()
y_hat = model(X_test)
